In [52]:
# Microsoft Threat Protection advanced hunting notebook

# Version 1.0
# Author: Maarten Goet, MVP & RD
# http://www.maartengoet.org 

import json
import pandas
import urllib.request
import urllib.parse

# Your MTP environment

tenantId = '00000000-0000-0000-0000-000000000000' # Replace with your Tenant ID
appId = '000000000000000000-0000-000000000000' # Replace with your Application ID
appSecret = '0000000000000000000000000000000000' # Replace with the Secret for your Application

url = "https://login.windows.net/%s/oauth2/token" % (tenantId)

resourceAppIdUri = 'https://api.security.microsoft.com' # Hello, MTP

body = {
    'resource' : resourceAppIdUri,
    'client_id' : appId,
    'client_secret' : appSecret,
    'grant_type' : 'client_credentials'
}

data = urllib.parse.urlencode(body).encode("utf-8")

req = urllib.request.Request(url, data)
response = urllib.request.urlopen(req)
jsonResponse = json.loads(response.read())
aadToken = jsonResponse["access_token"] # Access token for the next hour

In [53]:
# Specify your Advanced Hunting query (KQL)
# Sample: get indicators of comprise for COVID-19 campaigns

query = ('''

let Covid19IoC = (externaldata(Covid19Indicators:string )
[@"https://raw.githubusercontent.com/Azure/Azure-Sentinel/master/Sample%20Data/Feeds/Microsoft.Covid19.Indicators.json"] with (format="multijson"));
Covid19IoC
| mv-expand(parse_json(Covid19Indicators))
| project Covid19Indicators = todynamic(Covid19Indicators)
| evaluate bag_unpack(Covid19Indicators) | evaluate bag_unpack(AdditionalMetadata)
| project FirstSeen, ThreatType, FileSha256=Indicator, FileMd5, FileSha1, ExternalId

''') # KQL

# Need inspiration? Here's a MTP cheat sheet: 
# https://medium.com/threathunt/introducing-mtp-advance-hunting-cheat-sheet-1535862c5e84

In [50]:
url = "https://api.security.microsoft.com/api/advancedhunting/run" # Query the MTP Advanced Hunting API
headers = { 
	'Content-Type' : 'application/json',
	'Accept' : 'application/json',
	'Authorization' : "Bearer " + aadToken
}

data = json.dumps({ 'Query' : query }).encode("utf-8")

req = urllib.request.Request(url, data, headers)
response = urllib.request.urlopen(req)
jsonResponse = json.loads(response.read())
schema = jsonResponse["Schema"]
results = jsonResponse["Results"] # JSON response will be loaded in variable called 'results'

In [54]:
from pandas.io.json import json_normalize

json_normalize(results) # Convert JSON to a dataframe and display output

,FirstSeen,ThreatType,FileSha256,FileMd5,FileSha1,ExternalId
0,2020-07-29T00:17:55Z,Phish,1b0a0abb55a0d47ae9439947a022ccf4901876622a40ef...,afde4667948a55f8539aa9425c7f62ef,988c1e37a8c3e9045c582080035ab8f95b774451,58b808d3-30d1-ea11-b0a1-9cdc7141d162
1,2020-07-07T20:55:05Z,Malware,064147ab90d91d91a642163b45fc1db3b93f7a0ee9ea34...,070e8c59080ff8dadeb45e6f569185cb,93affa95cd2d4c62beb3115b039ae7e44a12ca1c,8089daad-93c0-ea11-b0a5-9cdc7148e892
2,2020-07-07T20:40:38Z,Malware,5f9a2e47545a9cafaefb332e7e1c4ae36d2fe7f0c04890...,500d9d25c3592ca97b0df084f9f5dbda,289991696ffc5bec3da311b7b809a4d976f68e4f,858a79a7-91c0-ea11-b0a8-040973d20fc2
3,2020-07-07T19:58:21Z,Malware,dc9124c7f6cc9a45e5c6330f16d52b6b279e654cd974b4...,f236dd1fd018d7b2514204177e7ae2d7,ec4508b08b023d9a454f5c92adecc5516a4528f2,251647c5-8bc0-ea11-b0aa-040973d37112
4,2020-07-07T20:11:08Z,Malware,e055b8ceea393ed64596f0398f33d06d863fdb3ef843f7...,ae24304e4d95499201e4a88577c17abe,bbb9f4e30ce3b75e938dc67cd939e24520610082,40747987-8dc0-ea11-b083-00155d012c08
...,...,...,...,...,...,...
357,2020-08-13T21:25:44Z,Phish,baca3c59494f002664d4993d683f190984ad8d4ba7705c...,ae3c4d5b384e72cf09d2412ce169f3aa,005616da7e72e428805f0e783e43dca8052c956a,405ad461-abdd-ea11-b0a5-9cdc71462bf2
358,2020-08-10T10:05:35Z,Phish,037c58735813a7a287793b74ca113b73bdc61a8657e723...,2082b9e191e5c929acc40a9a7d8392b6,2a3e2157a4351e53024275eb69db372a91e3b8ec,1a79a5f1-f0da-ea11-b0ac-9cdc7150a8b2
359,2020-08-09T19:30:26Z,Phish,6634cad00f3cf79d858882f2c1170bb9f6486c01f2e97c...,4c6785476f3931151cbe34cd11634f7f,93a4f074e6b5c10657b9075ebc4940e2dcf3695f,61ccca10-76da-ea11-b0aa-040973d2dbe2
360,2020-08-09T15:52:39Z,Phish,ffafe3c772ee00762745c0fdecfc910da06f043ef1cf96...,3ef12c53b77b13c179e8c91810850939,c6988d37407d2105997281d977b6333f3eeb9024,6e2c7e1a-58da-ea11-b09a-e0071b7ca442
